In [2]:
import os
import cv2
import numpy as np
import random
from math import pi, cos, sin, radians

# --- パラメータ一括管理 ---
CONFIG = {
    "img_size": 512,                     # 画像の幅・高さ（正方形）
    "num_images": 6000,                   # 生成する画像の枚数
    "max_shapes_per_image": 3,           # 1画像あたりの最大図形数

    # 頂点・座標揺らぎ
    "jitter_point": 3,                   # 円の点座標揺らぎ(px)
    "vertex_jitter_triangle_square": 15, # 三角形・四角形の頂点揺らし幅(px)

    # 円の設定
    "circle_points_num": 30,             # 円を構成する点の数（多いほど滑らか）
    "circle_radius_range": (20, 80),    # 円の半径の最小・最大値
    "circle_radius_jitter": 5,           # 円の半径を揺らす幅(px)
    "circle_rotation_angle_range": (-15, 15),  # 円の回転角度範囲（度数法）

    # 多角形の設定
    "polygon_size_range": (40, 150),     # 三角形・四角形のサイズ範囲
    "polygon_rotation_angle_range": (0, 180), # 多角形の回転角度範囲（度数法）

    # 線の設定
    "line_thickness_range": (2, 4),      # 通常線の太さ範囲（ランダム）
    "normal_line_color": 255,             # 通常の線の色（白）
}

def jitter_point(x, y, jitter):
    """指定した揺らし幅で座標をランダムにずらす"""
    return x + random.uniform(-jitter, jitter), y + random.uniform(-jitter, jitter)

def rotate_points(pts, angle_deg, center):
    """指定中心(center)を軸に点群ptsをangle_deg度回転させる"""
    angle_rad = radians(angle_deg)
    cos_a = cos(angle_rad)
    sin_a = sin(angle_rad)
    cx, cy = center
    new_pts = []
    for (x, y) in pts:
        tx, ty = x - cx, y - cy
        rx = tx * cos_a - ty * sin_a + cx
        ry = tx * sin_a + ty * cos_a + cy
        new_pts.append((rx, ry))
    return new_pts

def draw_handwritten_polygon(img, pts_np):
    """手書き風の多角形を描画（途切れ線なし）"""
    thickness_normal = random.randint(*CONFIG["line_thickness_range"])
    color_normal = CONFIG["normal_line_color"]
    cv2.polylines(img, [pts_np], isClosed=True, color=color_normal, thickness=thickness_normal)
    
def polygon_area(pts):
    """
    pts: 頂点リスト [(x1, y1), ..., (xn, yn)]
    多角形の面積をシューズレースの公式で計算
    """
    n = len(pts)
    area = 0
    for i in range(n):
        x1, y1 = pts[i]
        x2, y2 = pts[(i + 1) % n]
        area += x1 * y2 - x2 * y1
    return abs(area) / 2

def draw_handwritten_circle(img):
    """手書き風の円を描く"""
    r = random.randint(*CONFIG["circle_radius_range"])
    cx = random.randint(r, CONFIG["img_size"] - r)
    cy = random.randint(r, CONFIG["img_size"] - r)
    num_points = CONFIG["circle_points_num"]
    angle_step = 360 / num_points
    pts = []
    for i in range(num_points):
        angle = angle_step * i
        rad = radians(angle)
        radius_jitter = r + random.uniform(-CONFIG["circle_radius_jitter"], CONFIG["circle_radius_jitter"])  # 半径揺らぎ
        x = cx + radius_jitter * cos(rad)
        y = cy + radius_jitter * sin(rad)
        x, y = jitter_point(x, y, CONFIG["jitter_point"])  # 座標揺らぎ
        pts.append((x, y))

    angle_rot = random.uniform(*CONFIG["circle_rotation_angle_range"])
    pts = rotate_points(pts, angle_rot, (cx, cy))

    pts_np = np.array(pts, np.int32)
    thickness = random.randint(1, 3)

    cv2.polylines(img, [pts_np], isClosed=True, color=CONFIG["normal_line_color"], thickness=thickness)

    x, y, w, h = cv2.boundingRect(pts_np)
    area = pi * (r ** 2)  # 面積は理論値で計算

    return 0, [x, y, w, h], area

def draw_handwritten_triangle(img):
    """手書き風の三角形を描く"""
    size = random.randint(*CONFIG["polygon_size_range"])
    x = random.randint(0, CONFIG["img_size"] - size)
    y = random.randint(0, CONFIG["img_size"] - size)

    pts = [
        (x, y + size),
        (x + size / 2, y),
        (x + size, y + size)
    ]
    pts = [jitter_point(px, py, CONFIG["vertex_jitter_triangle_square"]) for (px, py) in pts]

    cx = sum(p[0] for p in pts) / 3
    cy = sum(p[1] for p in pts) / 3
    angle_rot = random.uniform(*CONFIG["polygon_rotation_angle_range"])
    pts = rotate_points(pts, angle_rot, (cx, cy))

    pts_np = np.array(pts, np.int32)

    draw_handwritten_polygon(img, pts_np)

    x_b, y_b, w, h = cv2.boundingRect(pts_np)
    x1, y1 = pts[0]
    x2, y2 = pts[1]
    x3, y3 = pts[2]
    area = abs((x1*(y2 - y3) + x2*(y3 - y1) + x3*(y1 - y2)) / 2)

    return 1, [x_b, y_b, w, h], area

def draw_handwritten_square(img):
    """手書き風の四角形を描く"""
    size = random.randint(*CONFIG["polygon_size_range"])
    x = random.randint(0, CONFIG["img_size"] - size)
    y = random.randint(0, CONFIG["img_size"] - size)

    pts = [
        (x, y),
        (x + size, y),
        (x + size, y + size),
        (x, y + size)
    ]
    pts = [jitter_point(px, py, CONFIG["vertex_jitter_triangle_square"]) for (px, py) in pts]

    cx = sum(p[0] for p in pts) / 4
    cy = sum(p[1] for p in pts) / 4
    angle_rot = random.uniform(*CONFIG["polygon_rotation_angle_range"])
    pts = rotate_points(pts, angle_rot, (cx, cy))

    pts_np = np.array(pts, np.int32)

    draw_handwritten_polygon(img, pts_np)

    x_b, y_b, w, h = cv2.boundingRect(pts_np)
    area = polygon_area(pts)

    return 2, [x_b, y_b, w, h], area

def normalize_bbox(bbox):
    x, y, w, h = bbox
    cx = (x + w / 2) / CONFIG["img_size"]
    cy = (y + h / 2) / CONFIG["img_size"]
    nw = w / CONFIG["img_size"]
    nh = h / CONFIG["img_size"]
    return cx, cy, nw, nh

draw_funcs = [draw_handwritten_circle, draw_handwritten_triangle, draw_handwritten_square]

# 保存ディレクトリ作成
path = r"C:\Users\micha\Desktop\Shape"
os.makedirs(os.path.join(path, "images"), exist_ok=True)
os.makedirs(os.path.join(path, "labels"), exist_ok=True)

for i in range(CONFIG["num_images"]):
    img = np.zeros((CONFIG["img_size"], CONFIG["img_size"]), dtype=np.uint8)
    label_lines = []
    num_shapes = random.randint(1, CONFIG["max_shapes_per_image"])
    for _ in range(num_shapes):
        shape_fn = random.choice(draw_funcs)
        class_id, bbox, area = shape_fn(img)
        cx, cy, w, h = normalize_bbox(bbox)
        area_norm = area / (CONFIG["img_size"] * CONFIG["img_size"])
        label_lines.append(f"{class_id} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f} {area_norm:.6f}")

    cv2.imwrite(os.path.join(path, f"images/img_{i:04d}.png"), img)
    with open(os.path.join(path, f"labels/img_{i:04d}.txt"), "w") as f:
        f.write("\n".join(label_lines))

print(f"{CONFIG['num_images']}枚の手書き風図形画像とラベルを生成しました！")

6000枚の手書き風図形画像とラベルを生成しました！


In [3]:
"""
images/ と labels/ を train / val に分けて dataset/ 以下に移動する簡易スクリプト
"""

# 設定
IMAGE_DIR = path + "\images"
LABEL_DIR = path + "\labels"
OUTPUT_DIR = path + "\dataset"
TRAIN_RATIO = 0.8  # train の割合（残りは val）

# 出力先フォルダを作成
for split in ["train", "val"]:
    os.makedirs(f"{OUTPUT_DIR}/{split}/images", exist_ok=True)
    os.makedirs(f"{OUTPUT_DIR}/{split}/labels", exist_ok=True)

# ファイル一覧を取得・シャッフル
image_files = sorted(f for f in os.listdir(IMAGE_DIR) if f.endswith(".png"))
random.shuffle(image_files)

# 分割処理
num_train = int(len(image_files) * TRAIN_RATIO)
splits = [("train", image_files[:num_train]), ("val", image_files[num_train:])]

for split_name, files in splits:
    for img_file in files:
        label_file = img_file.replace(".png", ".txt")

        os.rename(f"{IMAGE_DIR}/{img_file}", f"{OUTPUT_DIR}/{split_name}/images/{img_file}")
        os.rename(f"{LABEL_DIR}/{label_file}", f"{OUTPUT_DIR}/{split_name}/labels/{label_file}")

print(f"✅ データセットを分割しました：train={num_train} / val={len(image_files) - num_train}")

✅ データセットを分割しました：train=4800 / val=1200
